In [2]:
import os
from pathlib import Path
import pandas as pd
from collections import Counter, defaultdict

# Spalten Checken

In [15]:
sf_raw = Path("/workspaces/projekt-datascience/data/raw/montreal")
csv_files = sorted(sf_raw.glob("**/*.csv*"))
print(f"gefundene CSVs: {len(csv_files)}")

gefundene CSVs: 61


In [16]:
def read_columns_fastish(p):
    cols = pd.read_csv(p, nrows=0).columns
    raw_cols = list(cols)
    norm_cols = tuple(col.strip().lower() for col in cols)
    return raw_cols, norm_cols

In [17]:
schema_groups = defaultdict(list)
col_counter = Counter()
per_file_cols = {}

for f in csv_files:
    raw_cols, norm_cols = read_columns_fastish(f)
    schema_groups[norm_cols].append(f)
    col_counter.update(norm_cols)

In [18]:
print("Einzigartige Schemas")
for i, (schema, files) in enumerate(schema_groups.items(), start=1):
    print(f"\nSchema #{i} — {len(files)} Datei(en)")
    for ex in files:
        print(f"  • {ex.relative_to(sf_raw)}")
    print("Spalten:")
    for c in schema:
        print(f"  - {c}")

Einzigartige Schemas

Schema #1 — 7 Datei(en)
  • 2014/Stations_2014.csv
  • 2015/Stations_2015.csv
  • 2016/Stations_2016.csv
  • 2017/Stations_2017.csv
  • 2018/Stations_2018.csv
  • 2019/Stations_2019.csv
  • 2020/stations.csv
Spalten:
  - code
  - name
  - latitude
  - longitude

Schema #2 — 48 Datei(en)
  • 2014/monthly/OD_2014-04.csv
  • 2014/monthly/OD_2014-05.csv
  • 2014/monthly/OD_2014-06.csv
  • 2014/monthly/OD_2014-07.csv
  • 2014/monthly/OD_2014-08.csv
  • 2014/monthly/OD_2014-09.csv
  • 2014/monthly/OD_2014-10.csv
  • 2014/monthly/OD_2014-11.csv
  • 2015/monthly/OD_2015-04.csv
  • 2015/monthly/OD_2015-05.csv
  • 2015/monthly/OD_2015-06.csv
  • 2015/monthly/OD_2015-07.csv
  • 2015/monthly/OD_2015-08.csv
  • 2015/monthly/OD_2015-09.csv
  • 2015/monthly/OD_2015-10.csv
  • 2015/monthly/OD_2015-11.csv
  • 2016/monthly/OD_2016-04.csv
  • 2016/monthly/OD_2016-05.csv
  • 2016/monthly/OD_2016-06.csv
  • 2016/monthly/OD_2016-07.csv
  • 2016/monthly/OD_2016-08.csv
  • 2016/monthly/O

# San Fran in Parquet

In [1]:
from pathlib import Path
import re
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
RAW_CITY = Path("/workspaces/projekt-datascience/data/raw/san_francisco")
OUT_BASE = "/workspaces/projekt-datascience/data/parquet/bronze/usage"  # Hive-Partitionen city/year/month
CITY = "san_francisco"

In [3]:
spark = (SparkSession.builder
         .appName("bronze-parquet-bikes")
         .config("spark.sql.sources.partitionOverwriteMode", "dynamic")
         .config("spark.sql.parquet.compression.codec", "snappy")
         .getOrCreate())

JAVA_HOME is not set


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.